###Ingestion del archivo "language.csv"

In [0]:
%run "../includes/configuration"

In [0]:
%run "../includes/commom_functions"

In [0]:
dbutils.widgets.text("p_environment","")
v_environment = dbutils.widgets.get("p_environment")

In [0]:
dbutils.widgets.text("p_file_date","2024-12-30")
v_file_date = dbutils.widgets.get("p_file_date")

In [0]:
display(v_file_date)

####Paso 1 - Leer el archivo CSV usando "DataFrameReader" de Spark

In [0]:
from pyspark.sql.types import StructField, StructType, StringType, IntegerType

In [0]:
language_schema = StructType(fields=[
    StructField("languageId", IntegerType(), False),
    StructField("languageCode", StringType(), True),
    StructField("languageName", StringType(), True)
])


In [0]:
language_df = spark.read \
            .schema(language_schema) \
            .option("header", True) \
            .csv(f"{bronze_folder_path}/{v_file_date}/language.csv")

####Paso2- Seleccionar las columnas requeridas

In [0]:
from pyspark.sql.functions import col 

In [0]:
language_selected_df = language_df.select(col("languageid"),col("languagename"))

####Paso3 - Cambiar nombre de las columnas

In [0]:
language_renamed_df = language_selected_df.withColumnsRenamed({"languageid":"language_id","languagename":"language_name"})

####Paso4 - Agregar columnas al DF

In [0]:
from pyspark.sql.functions import current_timestamp, lit

In [0]:
language_final_df = add_ingestion_dat(language_renamed_df) \
                        .withColumn("environment",lit(v_environment)) \
                        .withColumn("file_date",lit(v_file_date))

####Paso 5 - Escribir datos en el DataLake en formato Parquet

In [0]:
overwrite_partition(language_final_df, "movie_silver", "languages", "file_date")

In [0]:
#language_final_df.write.mode("append").partitionBy("file_date").format("parquet").saveAsTable("movie_silver.language")
#language_final_df.write.mode("overwrite").format("parquet").saveAsTable("movie_silver.languages")
#language_final_df.write.mode("overwrite").parquet(f"{silver_folder_path}/languages/")
language_final_df.write.mode("append").partitionBy("file_date").format("delta").saveAsTable("movie_silver.language")

In [0]:
#language_final_df.write.mode("overwrite").parquet(f"{silver_folder_path}/languages/")
language_final_df.write.mode("append").partitionBy("file_date").parquet(f"{silver_folder_path}/languages")

In [0]:
%sql
select file_date, count(1) 
from movie_silver.language
group by file_date;

In [0]:
display(spark.read.parquet(f"{silver_folder_path}/languages/"))

In [0]:
%fs
ls /mnt/mymoviehistory/silver/languages/

In [0]:
display(spark.read.parquet("/mnt/mymoviehistory/silver/languages/"))

In [0]:
display(language_final_df)

In [0]:
display(language_renamed_df)

In [0]:
display(language_selected_df)

In [0]:
display(language_df)

In [0]:
dbutils.notebook.exit("Success")